In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough)
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires google-cloud-storage<4.0.0,>=2.18.0, but you have google-cloud-storage 1.43.0 which is incompatible.
bigframes 2.31.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.43.0 which is incompatible.


In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


--- STARTING PARALLEL INITIALIZATION (RAM OPTIMIZED) ---
COMPLETED: Titles: 6348910


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# uncomment the code below and execute to reload the module when you make
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(se)

Config loaded for Project: storied-epigram-480214-e9


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


--- STARTING PARALLEL INITIALIZATION (RAM OPTIMIZED) ---
COMPLETED: Titles: 6348910


<module 'search_frontend' from '/content/search_frontend.py'>

In [ ]:
################## Rons added cell #######################
# STOP the background process first
try:
    proc.terminate()
except:
    pass

# Run this to kill the "ghost" process holding the port
# This kills any hidden process holding onto Port 5000
!fuser -k 5000/tcp
#fuser -k 8080/tcp

# Run this to see the REAL error logs
# If it crashes here, you will see a red error message explaining why
#se.app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

5000/tcp:            112828


In [ ]:
# find Colab's public URL
from google.colab.output import eval_js
server_url = eval_js("google.colab.kernel.proxyPort(5000)")
print(f"""Test your search engine by navigating to
{server_url}search?query=hello+world
This URL is only accessible from the same browser session. In other words, this
will not be accessible from a different machine, browser, or incognito session.
""")

# Uncomment the following line of code to run the frontend in the main
# process and wait for HTTP requests (colab will hang). The debug parameter
# lets you see incoming requests and get debug print outs if exceptions occur.
# se.run(debug=False, use_reloader=False)

# Alternatively, the next few lines run the frontend in a background process.
# Just don't forget to terminate the process when you update your search engine
# or want to reload it.
import multiprocessing, time
proc = multiprocessing.Process(target=se.run,
                               kwargs={"debug": True, "use_reloader": False,
                                       "host": "0.0.0.0", "port": 5000})
proc.start()

time.sleep(1) # give Flask time to boot

from google.colab.output import eval_js
server_url = eval_js("google.colab.kernel.proxyPort(5000)")

print(f"Open this URL:\n{server_url}/search?query=hello+world")
# Use proc.terminate() to stop the process

Test your search engine by navigating to
https://5000-m-s-2urgxzxonbx3v-a.us-east4-1.prod.colab.devsearch?query=hello+world
This URL is only accessible from the same browser session. In other words, this
will not be accessible from a different machine, browser, or incognito session.

 * Serving Flask app 'search_frontend'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Open this URL:
https://5000-m-s-2urgxzxonbx3v-a.us-east4-1.prod.colab.dev/search?query=hello+world


# Testing your app

Once your app is running you can query it. You can simply do that by clicking on the URL printed above (the one looking like https://XXXXX-5000-colab.googleusercontent.com/search?query=hello+world or by issuing an HTTP request through code (from colab).

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
def precision_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(predicted_list) == 0:
        return 0.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(predicted_list), 3)
def recall_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(true_set) < 1:
        return 1.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(true_set), 3)
def f1_at_k(true_list, predicted_list, k):
    p = precision_at_k(true_list, predicted_list, k)
    r = recall_at_k(true_list, predicted_list, k)
    if p == 0.0 or r == 0.0:
        return 0.0
    return round(2.0 / (1.0/p + 1.0/r), 3)
def results_quality(true_list, predicted_list):
    p5 = precision_at_k(true_list, predicted_list, 5)
    f1_30 = f1_at_k(true_list, predicted_list, 30)
    if p5 == 0.0 or f1_30 == 0.0:
        return 0.0
    return round(2.0 / (1.0/p5 + 1.0/f1_30), 3)

assert precision_at_k(range(10), [1,2,3] , 2) == 1.0
assert recall_at_k(   range(10), [10,5,3], 2) == 0.1
assert precision_at_k(range(10), []      , 2) == 0.0
assert precision_at_k([],        [1,2,3],  5) == 0.0
assert recall_at_k(   [],        [10,5,3], 2) == 1.0
assert recall_at_k(   range(10), [],       2) == 0.0
assert f1_at_k(       [],        [1,2,3],  5) == 0.0
assert f1_at_k(       range(10), [],       2) == 0.0
assert f1_at_k(       range(10), [0,1,2],  2) == 0.333
assert f1_at_k(       range(50), range(5), 30) == 0.182
assert f1_at_k(       range(50), range(10), 30) == 0.333
assert f1_at_k(       range(50), range(30), 30) == 0.75
assert results_quality(range(50), range(5))  == 0.308
assert results_quality(range(50), range(10)) == 0.5
assert results_quality(range(50), range(30)) == 0.857
assert results_quality(range(50), [-1]*5 + list(range(5,30))) == 0.0


In [ ]:
import requests
import json
import pandas as pd
from time import time

def run_full_system_test(queries_file, ids_file):
    base_url = 'http://127.0.0.1:5000'

    # --- PART 1: SEARCH EVALUATION ---
    print(f"\n{'PART 1: SEARCH EVALUATION':^75}")
    print(f"{'Query':<40} | {'P@10':<6} | {'Qual':<6} | {'Time(s)':<8}")
    print("-" * 75)

    with open(queries_file, 'r') as f:
        queries = json.load(f)

    search_stats = []
    for q, true_wids in queries.items():
        t_start = time()
        try:
            res = requests.get(f"{base_url}/search", {'query': q}, timeout=300)
            duration = time() - t_start

            p10, p5, f1_30, qual = 0.0, 0.0, 0.0, 0.0
            if res.status_code == 200:
                data = res.json()
                results_list = data["results"] if isinstance(data, dict) else data

                if results_list:
                    pred_wids = [str(item[0]) for item in results_list]
                    # Calculate all required metrics
                    p10 = precision_at_k(true_wids, pred_wids, 10)
                    p5 = precision_at_k(true_wids, pred_wids, 5)
                    f1_30 = f1_at_k(true_wids, pred_wids, 30)
                    qual = results_quality(true_wids, pred_wids)

                print(f"{q[:40]:<40} | {p10:<6.3f} | {qual:<6.3f} | {duration:<8.3f}")
            else:
                print(f"Error {res.status_code} for query '{q}'")
        except Exception as e:
            print(f"Failed query '{q}': {e}")
            duration = 0.0
            p10, p5, f1_30, qual = 0.0, 0.0, 0.0, 0.0

        search_stats.append({
            'Query': q,
            'P@10': p10,
            'P@5': p5,
            'F1@30': f1_30,
            'Quality': qual,
            'Time': duration
        })
    if search_stats:
        df = pd.DataFrame(search_stats)
        print("\n" + "="*50)
        print("FINAL SEARCH SUMMARY (MEAN SCORES)")
        print("="*50)
        print(f"Mean Precision@5:      {df['P@5'].mean():.4f}")
        print(f"Mean Precision@10:     {df['P@10'].mean():.4f}")
        print(f"Mean F1@30:            {df['F1@30'].mean():.4f}")
        print(f"Mean Results Quality:  {df['Quality'].mean():.4f}")
        print(f"Mean Latency (s):      {df['Time'].mean():.4f}")
        print("="*50)
    # --- PART 2: DATA RETRIEVAL LATENCY ---
    print(f"\n{'PART 2: DATA RETRIEVAL LATENCY (GCS ON-DEMAND)':^75}")
    print("-" * 75)

    with open(ids_file, 'r') as f:
        test_ids = json.load(f).get("test_ids", [])
    if len(test_ids) < 1000:
        test_ids += list(range(10000, 11000 - len(test_ids)))
    test_ids = test_ids[:1000]

    endpoints = ['/get_pagerank', '/get_pageview']
    data_stats = {}

    for ep in endpoints:
        t_start = time()
        res = requests.post(f"{base_url}{ep}", json=test_ids)
        duration = time() - t_start
        print(f"Endpoint: {ep:<15} | IDs: 1000  | Time: {duration:.4f}s")
        data_stats[ep] = duration

    # Display data retrieval stats as well
    print(f"PageRank Fetch (1k):   {data_stats.get('/get_pagerank'):.4f}s")
    print(f"PageView Fetch (1k):   {data_stats.get('/get_pageview'):.4f}s")
    print("="*50)

if __name__ == "__main__":
    run_full_system_test('queries_train.json', 'test_ids.json')


                         PART 1: SEARCH EVALUATION                         
Query                                    | P@10   | Qual   | Time(s) 
---------------------------------------------------------------------------
Mount Everest climbing expeditions       | 0.500  | 0.340  | 2.113   
Great Fire of London 1666                | 0.800  | 0.606  | 1.088   
Nanotechnology materials science         | 0.900  | 0.615  | 0.890   
Fossil fuels climate change              | 0.500  | 0.320  | 0.563   
DNA double helix discovery               | 0.500  | 0.475  | 0.341   
Printing press invention Gutenberg       | 0.800  | 0.555  | 0.558   
Ancient Egypt pyramids pharaohs          | 0.600  | 0.571  | 0.961   
Gothic literature Mary Shelley           | 0.200  | 0.197  | 1.191   
Robotics automation industry             | 0.700  | 0.493  | 0.822   
Television invention broadcast media     | 0.600  | 0.323  | 1.594   
Wright brothers first flight             | 0.900  | 0.604  | 1.293   
Steam l

In [ ]:
######## TESTING HYPER PARAMETERS ###########
import requests
import json
import pandas as pd
from time import time
import itertools
def run_hyperparameter_grid_search(queries_file):
    base_url = 'http://127.0.0.1:5000/search'

    with open(queries_file, 'r') as f:
        queries = json.load(f)

    # Define the ranges you want to test
    param_grid = {
        'pool': [70, 100],
        'w_title': [0.7, 0.8],
        'k1': [1.5],
        'stem_w': [0.5], # Testing how much weight to give to stemmed words
        'threshold' : [400000, 500000]
    }

    # Generate all possible combinations using itertools
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    overall_results = []

    for combo in combinations:
        print("\n" + "="*80)
        print(f"TESTING PARAMETERS: {combo}")
        print("="*80)
        print(f"{'Query':<40} | {'P@10':<6} | {'Qual':<6} | {'Time(s)':<8}")
        print("-" * 75)

        iteration_stats = []

        for q, true_wids in queries.items():
            # Inject parameters into the request
            params = {'query': q, **combo}

            t_start = time()
            try:
                res = requests.get(base_url, params=params, timeout=300)
                duration = time() - t_start

                # Metrics for this query
                p10, p5, f1_30, qual = 0.0, 0.0, 0.0, 0.0

                if res.status_code == 200:
                    data = res.json()
                    results = data.get("results", [])

                    if results:
                        pred_wids = [str(item[0]) for item in results]
                        # Use your pre-defined functions
                        p10 = precision_at_k(true_wids, pred_wids, 10)
                        p5 = precision_at_k(true_wids, pred_wids, 5)
                        f1_30 = f1_at_k(true_wids, pred_wids, 30)
                        qual = results_quality(true_wids, pred_wids)

                    print(f"{q[:40]:<40} | {p10:<6.3f} | {qual:<6.3f} | {duration:<8.3f}")

                iteration_stats.append({
                    'P@5': p5, 'P@10': p10, 'F1@30': f1_30, 'Quality': qual, 'Time': duration
                })
            except Exception as e:
                print(f"Query {q} failed: {e}")

        # Summary for this combination
        df = pd.DataFrame(iteration_stats)
        mean_qual = df['Quality'].mean()

        print("\n" + "FINAL SEARCH SUMMARY (MEAN SCORES)")
        print("="*50)
        print(f"Mean Precision@5:      {df['P@5'].mean():.4f}")
        print(f"Mean Precision@10:     {df['P@10'].mean():.4f}")
        print(f"Mean F1@30:            {df['F1@30'].mean():.4f}")
        print(f"Mean Results Quality:  {mean_qual:.4f}")
        print(f"Mean Latency (s):      {df['Time'].mean():.4f}")
        print("="*50)

        overall_results.append({
            'params': combo,
            'mean_quality': mean_qual
        })

    # --- SHOW THE BEST COMBINATION ---
    best_test = max(overall_results, key=lambda x: x['mean_quality'])
    print("\n" + "*"*80)
    print(f"HIGHEST MEAN RESULTS QUALITY: {best_test['mean_quality']:.4f}")
    print(f"BEST HYPER-PARAMETERS: {best_test['params']}")
    print("*"*80 + "\n")

if __name__ == "__main__":
    run_hyperparameter_grid_search('queries_train.json')


TESTING PARAMETERS: {'pool': 70, 'w_title': 0.7, 'k1': 1.5, 'stem_w': 0.5, 'threshold': 400000}
Query                                    | P@10   | Qual   | Time(s) 
---------------------------------------------------------------------------
Mount Everest climbing expeditions       | 0.400  | 0.312  | 0.406   
Great Fire of London 1666                | 0.800  | 0.578  | 0.436   
Nanotechnology materials science         | 0.800  | 0.548  | 0.778   
Fossil fuels climate change              | 0.600  | 0.320  | 0.517   
DNA double helix discovery               | 0.500  | 0.447  | 0.348   
Printing press invention Gutenberg       | 0.900  | 0.555  | 0.531   
Ancient Egypt pyramids pharaohs          | 0.800  | 0.571  | 0.704   
Gothic literature Mary Shelley           | 0.200  | 0.222  | 0.778   
Robotics automation industry             | 0.700  | 0.493  | 0.623   
Television invention broadcast media     | 0.500  | 0.290  | 1.016   
Wright brothers first flight             | 1.000  | 0.549